## Часть 1 - Парсинг с сайта Streamersbase.ru

### Часть 1 - Парсинг с сайта Streamersbase.ru

In [ ]:
import requests
import lxml
from bs4 import BeautifulSoup
import json
import time
import csv
import random
import pandas as pd

#### 1. Напишем такой цикл, чтобы каждая страница сайта (на которой несколько карточек стримеро), сохранилась отдельным html (всего 138 страниц).
#### 2. Затем на каждой странице будем искать ссылку на карточки стримера (она находите в классе "streamer-name" и записывать в json
#### 3. Загрузим это в переменную с которой потом будем работать
#### 4. Для удобства отобразим, сколько их всео

In [94]:
all_streamers_list = []
count = 0

for i in range(1,139):
    
    url = "https://streamersbase.ru/streamers?page=" + str(i)
    
    headers = {
    "accept": "*/*",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36"
}
    req = requests.get(url, headers=headers)
    src = req.text
    
    with open(f"data/pages/{i}_index.html", "w") as file:
        file.write(src)
    
    with open(f"data/pages/{i}_index.html") as file:
        src = file.read()
    
    soup = BeautifulSoup(src, "lxml")
    
    all_streamers_hrefs = soup.find_all(class_="streamer-name")
    

    for item in all_streamers_hrefs:
        
        item_href = "https://streamersbase.ru" + item.get("href")
        all_streamers_list.append(item_href)

    count += 1

with open("data/all_streamers_list.json", "w") as file:
    json.dump(all_streamers_list, file, indent=4, ensure_ascii=False) 

with open("data/all_streamers_list.json") as file:
    all_streamers = json.load(file)
    
    
print(f"Всего карточек стримеров: {len(all_streamers_list)}")
print(f"Всего страниц сайта: {count}")

Всего карточек стримеров: 2755
Всего страниц сайта: 138


#### Далее сделаем цикл, в котором:
#### 1. Соберем названия столбцов с страницы стримера-1 (с данными, которые потом будем собирать) и запишем их в файл
#### 2. Теперь соберем необходимые данные и допишем в файл
#### 3. Далее будем дописывать новые данные в уже созданный файлю

In [102]:
iteration_count = len(all_streamers)
count = 0

print(f"Всего итераций: {iteration_count}")

for streamer_href in all_streamers_list:

    req = requests.get(url=streamer_href, headers=headers)
    src = req.text
    time.sleep(1)

    with open(f"data/streamers_htmls/{count}.html", "w") as file:
        file.write(src)

    with open(f"data/streamers_htmls/{count}.html") as file:
        src = file.read()

    soup = BeautifulSoup(src, "lxml")

    # Соберем названия нужных столбцов 
    table_about = soup.find_all(class_="table bio-table")[0].find_all(class_="text-muted")
    streamer_gender = table_about[1].text.strip()
    streamer_age = table_about[3].text.strip()

    table_social = soup.find_all(class_="table bio-table")[1].find_all(class_="text-muted")
    telegram = table_social[0].text.strip()
    vkontakte = table_social[1].text.strip()
    instagram = table_social[2].text.strip()
    tiktok = table_social[3].text.strip()
    facebook = table_social[4].text.strip()
    youtube = table_social[5].text.strip()
    twitter = table_social[6].text.strip()
    discord = table_social[7].text.strip()
    steam = table_social[8].text.strip()
    twitch = table_social[9].text.strip()
    email = table_social[10].text.strip()
    
    # На первой итерации запишем в файл названия столбцов, чтобы русские символы отображались добави utf-8 кодировку
    if count < 1:
        
        print("ФАЙЛ ЗАПИСЫВАЕТСЯ")
        with open(f"data/all_streamers_data.csv", "w",  encoding="utf-8") as file:
            writer = csv.writer(file, delimiter=",")
            writer.writerow(
                (
                    streamer_gender,
                    streamer_age,
                    telegram,
                    vkontakte,
                    instagram,
                    tiktok,
                    facebook,
                    youtube,
                    twitter,
                    discord,
                    steam,
                    twitch,
                    email,  
                )

            )
        
    # Соберем данные стримеров по каждому из стобцов, создадим словарь для данных, куда будем их записывать
    streamer_data = []

    # Т.к. нам нужны не все из этой таблцы выберем конкретные и добавив в словарь (как нашли в нижнем чанке)
    streamer_data_about = soup.find_all(class_="table bio-table")[0].find_all(class_="text-fw-500")
    streamer_data.append(streamer_data_about[1].text)
    streamer_data.append(streamer_data_about[3].text.strip())

    # Из этой таблицы нужны все данные - пройдем циклом, добавляя в словарь
    streamer_data_social = soup.find_all(class_="table bio-table")[1].find_all(class_="text-fw-500")
    
    for i in streamer_data_social:
        
        streamer_data.append(i.text.strip())
           
    # Будем апендить уже созданный ранее файл и записывать данные стримеров
    with open("data/all_streamers_data.csv", "a",  encoding="utf-8") as file:
        writer = csv.writer(file, delimiter=",")
        writer.writerow(
            (
                 streamer_data
            )

        )

    # Для отслеживания прогресса будем выводить статус. Также увеличивать кантер на 1, чтобы двигаться по стримерам.    
    count += 1
    
    # Будем выводить статус по кол-ву собранных стримеров в разрезе 100
    if count % 100 == 0:
        
        print(count, end=",")
    
    iteration_count = iteration_count - 1

    # Выведем информацию о завершении
    if iteration_count == 0:
        
        print("Работа завершена")
        break

    time.sleep(random.randrange(1, 2))

Всего итераций: 2755
ФАЙЛ ЗАПИСЫВАЕТСЯ
100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300,2400,2500,2600,2700,Работа завершена


## Как были найдены необходимые столбцы?

In [122]:
# С помощью цикла посмотрим, какой элемент словаря отвечает за какое поле в таблице общее и выберем нужные: 1, 3 (пол и возраст)

counts = 0

for i in table_about:
    print(f"{counts}: {i.text}")
    
    counts += 1

0: Настоящее имя s1mple:
1: Пол:
2: Дата рождения:
3: Возраст:
4: Рост:
5: Знак зодиака:
6: Место рождения:
7: Место проживания:
8: Семейное положение:
9: Образование:


In [131]:
# Аналогично для таблицы социальных сетей - возьмем все.

counts = 0

for i in table_social:
    print(f"{counts}: {i.text}")
    
    counts += 1

0: Telegram:
1: ВКонтакте:
2: Instagram:
3: TikTok:
4: Facebook:
5: YouTube:
6: Twitter:
7: Discord:
8: Steam:
9: Twitch:
10: E-mail:


In [10]:
# Загрузим получвшийся файл в пандас

df = pd.read_csv("data/all_streamers_data.csv")

In [11]:
df.head()

,Пол:,Возраст:,Telegram:,ВКонтакте:,Instagram:,TikTok:,Facebook:,YouTube:,Twitter:,Discord:,Steam:,Twitch:,E-mail:
0,Мужской,23 года,Неизвестно,vk.com/s1mpjl3,www.instagram.com/s1mpleO,Неизвестно,Неизвестно,www.youtube.com/channel/UCWrZWoNJaF-pkkLWmg3Uzjg,twitter.com/s1mpleo,Неизвестно,steamcommunity.com/id/officials1mple,twitch.tv/s1mple,Неизвестно
1,Мужской,24 года,Неизвестно,vk.com/lilbuster,www.instagram.com/busterzy,www.tiktok.com/@busterenko_,Неизвестно,www.youtube.com/channel/UCve3ohcLCBO5uFoQcFGZMPQ,Неизвестно,Неизвестно,steamcommunity.com/id/slavabuster,twitch.tv/buster,"buster@streamers-alliance.gg, busterpwnz@gmai..."
2,Мужской,24 года,Неизвестно,vk.com/evelone,www.instagram.com/evelone192,Неизвестно,Неизвестно,www.youtube.com/evelone192,Неизвестно,discordapp.com/invite/evelone,steamcommunity.com/groups/Evelonegroup,twitch.tv/evelone192,kozakovdelovoy@yandex.ua
3,Мужской,23 года,Неизвестно,vk.com/flamesv,instagram.com/br4tishkin,Неизвестно,Неизвестно,www.youtube.com/c/bratishkinoff,Неизвестно,discord.gg/89squad,steamcommunity.com/id/FlameSV,twitch.tv/bratishkinoff,bratishkinoff@mail.ru
4,Студия,Неизвестно,Неизвестно,vk.com/starladdercsgo,instagram.com/starladdercsgo,Неизвестно,facebook.com/starladdercsgo,youtube.com/starladdercsgo,twitter.com/starladdercsgo,Неизвестно,steamcommunity.com/groups/StarLadder5,twitch.tv/starladder5,Неизвестно


In [12]:
# Наведем красоту - уберем двоеточия из названий столбцов

df.columns = df.columns.str.replace(":", "", regex=True)
df.head()

,Пол,Возраст,Telegram,ВКонтакте,Instagram,TikTok,Facebook,YouTube,Twitter,Discord,Steam,Twitch,E-mail
0,Мужской,23 года,Неизвестно,vk.com/s1mpjl3,www.instagram.com/s1mpleO,Неизвестно,Неизвестно,www.youtube.com/channel/UCWrZWoNJaF-pkkLWmg3Uzjg,twitter.com/s1mpleo,Неизвестно,steamcommunity.com/id/officials1mple,twitch.tv/s1mple,Неизвестно
1,Мужской,24 года,Неизвестно,vk.com/lilbuster,www.instagram.com/busterzy,www.tiktok.com/@busterenko_,Неизвестно,www.youtube.com/channel/UCve3ohcLCBO5uFoQcFGZMPQ,Неизвестно,Неизвестно,steamcommunity.com/id/slavabuster,twitch.tv/buster,"buster@streamers-alliance.gg, busterpwnz@gmai..."
2,Мужской,24 года,Неизвестно,vk.com/evelone,www.instagram.com/evelone192,Неизвестно,Неизвестно,www.youtube.com/evelone192,Неизвестно,discordapp.com/invite/evelone,steamcommunity.com/groups/Evelonegroup,twitch.tv/evelone192,kozakovdelovoy@yandex.ua
3,Мужской,23 года,Неизвестно,vk.com/flamesv,instagram.com/br4tishkin,Неизвестно,Неизвестно,www.youtube.com/c/bratishkinoff,Неизвестно,discord.gg/89squad,steamcommunity.com/id/FlameSV,twitch.tv/bratishkinoff,bratishkinoff@mail.ru
4,Студия,Неизвестно,Неизвестно,vk.com/starladdercsgo,instagram.com/starladdercsgo,Неизвестно,facebook.com/starladdercsgo,youtube.com/starladdercsgo,twitter.com/starladdercsgo,Неизвестно,steamcommunity.com/groups/StarLadder5,twitch.tv/starladder5,Неизвестно


In [14]:
# Посмотрим возраст

df["Пол"].value_counts() , df["Пол"].value_counts(normalize=True).round(2)

(Неизвестно    1722
 Мужской        626
 Женский        338
 Студия          69
 Name: Пол, dtype: int64,
 Неизвестно    0.63
 Мужской       0.23
 Женский       0.12
 Студия        0.03
 Name: Пол, dtype: float64)

In [15]:
# Видим, что с возрастом беда

df["Возраст"].value_counts()

Неизвестно    2332
27 лет          40
24 года         37
22 года         35
26 лет          34
28 лет          33
29 лет          31
25 лет          30
23 года         25
30 лет          23
31 год          22
33 года         20
21 год          18
32 года         18
20 лет          15
19 лет          10
34 года          7
18 лет           5
17 лет           4
35 лет           4
39 лет           3
36 лет           2
38 лет           2
1 год            1
41 год           1
42 года          1
55 лет           1
16 лет           1
Name: Возраст, dtype: int64

In [16]:
# Приведем возраст к одному виду - цифры года

df['Возраст'] = df['Возраст'].str.replace("лет", "", regex=True)
df['Возраст'] = df['Возраст'].str.replace("года", "", regex=True)
df['Возраст'] = df['Возраст'].str.replace("год", "", regex=True)
df['Возраст'] = df['Возраст'].str.replace(" ", "", regex=True)

df["Возраст"].value_counts()

Неизвестно    2332
27              40
24              37
22              35
26              34
28              33
29              31
25              30
23              25
30              23
31              22
33              20
32              18
21              18
20              15
19              10
34               7
18               5
17               4
35               4
39               3
36               2
38               2
55               1
16               1
41               1
1                1
42               1
Name: Возраст, dtype: int64

In [60]:
# Но есть возраст "1" - добавим его в неизвестно для чистоты

df[df["Возраст"] == "1"]

,Пол,Возраст,Telegram,ВКонтакте,Instagram,TikTok,Facebook,YouTube,Twitter,Discord,Steam,Twitch,E-mail
70,Мужской,1,Неизвестно,vk.com/teamnespit,www.instagram.com/spt083/,Неизвестно,Неизвестно,www.youtube.com/channel/UCuNB26UWfaQfF4ytLO84G...,Неизвестно,discordapp.com/invite/nespit,steamcommunity.com/id/spt083/,twitch.tv/spt083,Неизвестно


In [63]:
df.iloc[70, 1] = "Неизвестно"

In [92]:
# Сохраним итоговый файл

df.to_csv('data/all_streamers_data_final.csv', index=False)

## Часть 2 - Парсинг данных пользователей VK

## Перед этим проверим данные с сайта на корректность и че как

In [25]:
import re
import numpy as np

In [19]:
vkgroups = pd.read_csv("data/all_streamers_data_final.csv")
vkgroups.shape

(2755, 13)

In [20]:
vkgroups['ВКонтакте'].value_counts()

Неизвестно               1953
vk.com/ruslanabeylz         2
vk.com/m69bhrm              2
vk.com/babamtv              2
vk.com/taersclub            2
                         ... 
vk.com/ueber_marginal       1
vk.com/id139431539          1
vk.com/therockarmy          1
vk.com/gamelifeow           1
vk.com/streamann            1
Name: ВКонтакте, Length: 795, dtype: int64

In [23]:
# Проверим сходятся ли цифры

print("Не указана группа ВК:", vkgroups[vkgroups['ВКонтакте'] == "Неизвестно"].loc[:, "ВКонтакте"].count())
print("Указана группа ВК:", vkgroups[vkgroups['ВКонтакте'] != "Неизвестно"].loc[:, "ВКонтакте"].count())

Не указана группа ВК: 1953
Указана группа ВК: 802


In [7]:
# Видим, что все ок

1953 + 802

2755

In [8]:
# Создадим отдельный дата фрейм с ссылками на группы

vkgroups_group_link = pd.DataFrame()
vkgroups_group_link["group_link"] = vkgroups[vkgroups['ВКонтакте'] != "Неизвестно"].loc[:, "ВКонтакте"]
vkgroups_group_link = vkgroups_group_link.dropna()

In [9]:
vkgroups_group_link

,group_link
0,vk.com/s1mpjl3
1,vk.com/lilbuster
2,vk.com/evelone
3,vk.com/flamesv
4,vk.com/starladdercsgo
...,...
2700,vk.com/kate_mercy_stream
2705,vk.com/iriska_ira91
2707,vk.com/satanaqqpwnz_stream
2744,vk.com/s1rozha


# На этом этапе экспертным взглядом пробежались по списку:
## 1. Оказалось, что некоторые ссылки ведут на личные страницы, а не сообщества в ВК, поэтому добавили в цикл условие (будем печатать error)
## 2. По той-же причине было решено экспертно добавить ТОП-10 стримеров ру сегмента (перед этим проверив, есть ли они в списке) (взяли отсюда https://cq.ru/articles/streaming/top-10-russkoiazychnykh-strimerov-2020-na-twitch) и несколько своих любимых :)
## 3. Также экспертным взглядом прошлись по списку сообществ - удалили сообщества популярныx певцов (Крид и Тклила), т. к. они не постоянные стримеры и их аудитория даст большое искажение на данные (т.к. они большие)

### Поэтому выгрузим список и вручную проверим ссылки на первой странице и заменим личные на сообщества, а также сделаем пункт 2 и 3

In [49]:
# пример личной страницы

vkgroups_group_link.head(1)

,group_link
0,vk.com/s1mpjl3


In [31]:
# Сохраним файл для ручной правки

vkgroups_group_link.to_csv("data/vkgroups_group_link.csv", index=False)

In [50]:
# загрузим получившийся файл

vkgroups_group_link_modified = pd.read_csv("data/vkgroups_group_link_modified.csv")

In [51]:
# На данном этапе всего 807 групп (скорее всего еще часть из них личные страницы)

vkgroups_group_link_modified

,group_link
0,https://vk.com/morethanthree
1,https://vk.com/publicvoodoosh
2,https://vk.com/cakestream
3,https://vk.com/vsemilyubim
4,https://vk.com/showram
...,...
802,vk.com/kate_mercy_stream
803,vk.com/iriska_ira91
804,vk.com/satanaqqpwnz_stream
805,vk.com/s1rozha


In [52]:
# уберем лишнюю часть ссылки, оставив только название группы

vkgroups_group_link_clear = pd.DataFrame()

vkgroups_group_link_clear['group_link'] = vkgroups_group_link_modified['group_link'].str.replace('https://vk.com/','',regex=True)
vkgroups_group_link_clear['group_link'] = vkgroups_group_link_clear['group_link'].str.replace('http://vk.com/','',regex=True)
vkgroups_group_link_clear['group_link'] = vkgroups_group_link_clear['group_link'].str.replace('vk.com/','',regex=True)
vkgroups_group_link_clear['group_link'] = vkgroups_group_link_clear['group_link'].str.replace('www.','',regex=True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [63]:
group_list

['morethanthree', 'publicvoodoosh']

In [60]:
# все группы уникальны, всего 807 штук

vkgroups_group_link_clear['group_link'].unique

<bound method Series.unique of 0            morethanthree
1           publicvoodoosh
2               cakestream
3              vsemilyubim
4                  showram
              ...         
802      kate_mercy_stream
803           iriska_ira91
804    satanaqqpwnz_stream
805                s1rozha
806                satorik
Name: group_link, Length: 807, dtype: object>

In [ ]:
# создадим лист, в котором будут содержаться названия групп, с которыми далее будем работать (для начала выберем первые два в списке, чтобы не ждать овертайм)

group_list = pdToList = list(vkgroups_group_link_clear['group_link'][0:2])

# Работа с API_VK
### На этом этапе засетапим парсер данных пользователей из ранее созданного листа. Для его работы нужен будет лист, содержащий в себе очищенные названия групп. При желании им можно будет парсить любой набор групп для дальнеших задач и получать: id/страну/город/дату рождения/пол, либо информацию о том, что они не указаны, c условием, что пользователь заходил в ВК после какой-то даты (для отсеивания ботов и т.д.)

In [24]:
# установим библиотеку для работы с API ВК

import vk_api

In [26]:
# Тут нужно указать свой логин и пароль

login = "логин"
password = "пароль" 

In [27]:
# Зайдем в ВК

vk_session = vk_api.VkApi(login, password)
vk_session.auth()

vk_api = vk_session.get_api()

In [28]:
# Напишем функцию, которая будет высчитыват необходимое кол-во "отступов" от данных (т. к. присуствует ограничение на один вызов в 1к) для конкретной группы

def get_offset(group_id):
    count = vk_api.groups.getMembers(group_id = group_id,
                                     v=5.103, 
                                     offset=0)
    return count['count'] // 1000

In [31]:
# Пример в группе "morethanthree" на текущий момент 12864 подписчиков.

get_offset('morethanthree')

12

### То есть, чтобы выгрузить все данные нужно будет: 
### 1. Cпарсить первые 1000
### 2. Сделать отступ в 1000 и начать следующюю итерацию парсинга с 1001 до 2000
### 3. Сделать отступ в 1000 и начать с 3001 до 4000
### 4. И так далее столько раз, сколько мы получаем и написаной функции

In [7]:
# Напишем функцию, которая из указаной группы будет собирать необходимые нам данные, либо указывать, что они не указаны.

def get_users(group_id):
    # Создадим лист, куда будем собирать данные и укажем стартовый оффсет - 0, с помощью нашей функции высчитаем максимальный
    good_id_list = []
    offset = 0
    max_offset = get_offset(group_id) * 1000
    # Будем продолжать работу, пока текущий оффсет не будет равен максимальному. Сделаем запрос к группе и достанем необходимые данные
    while offset <= max_offset:
        response = vk_api.groups.getMembers(group_id = group_id,
                                            v=5.103,
                                            sort='id_desc',
                                            offset=offset,
                                            fields=['last_seen', 'sex', 'bdate', 'country', 'city'])
        # Прибивим в каунтер офсет
        offset += 1000
        time.sleep(3)
        №
        for item in response['items']:
            try:
                
                #Условие, что пользователь заходил в ВК после 1 декабря 2020 - отсеим ботов и все такое
                if item['last_seen']['time'] >= 1606780800:
                  #and item['country']['title'] == 1:      включить для сбора только РФ
                    good_id_list.append(item['id'])
                    # Для каждых пользователей будем пытаться достать необходимые данные и записывать их отсутсвие, если не выйдет
                    try:
                        good_id_list.append(item['sex'])
                    except:
                        good_id_list.append('Unfiled_sex')
                    
                    try:
                        good_id_list.append(item['country']['title'])
                    except:
                        good_id_list.append('Unfiled_country')
                    
                    try:
                        good_id_list.append(item['city']['title'])
                    except:
                        good_id_list.append('Unfiled_city')
                        
                    try:
                        good_id_list.append(item['bdate'])
                    except:
                        good_id_list.append('Unfiled_bdate')
                              
            except:
                continue
                
    return good_id_list

In [42]:
# Засетапим каунтер и список, в который будем записывать собранные данные

all_users = []
counter = 1

In [64]:
%%time

# all_users = []
# counter = 1

for group in group_list:
    
    try:
        users = get_users(group)
        all_users.extend(users)
        print(counter, group, end=", ")
        time.sleep(10)
        
    except:
        try:
            time.sleep(10)
            users = get_users(group)
            all_users.extend(users)
            print(counter, group, end=", ")
        
        except:
            print(counter, group, "ERROR", end=", ")
            continue
        
    counter += 1

2 morethanthree, 3 publicvoodoosh, CPU times: user 452 ms, sys: 44.2 ms, total: 496 ms
Wall time: 1min 52s


In [66]:
# сделаем датафрейм и раскидаем по столбцам. Также переведем названия полов в цифры и укажем года рождения если есть, если нет - укажем эту инфу

%time

all_users_total = pd.DataFrame()

all_users_total['id'] = all_users[0::5]
all_users_total['sex'] = all_users[1::5]
all_users_total['country'] = all_users[2::5]
all_users_total['city'] = all_users[3::5]
all_users_total['bdate'] = all_users[4::5]

all_users_total['sex'] = all_users_total['sex'].replace(0, "Unfiled_sex")
all_users_total['sex'] = all_users_total['sex'].replace(1, "Female")
all_users_total['sex'] = all_users_total['sex'].replace(2, "Male")

all_users_total['bdate'] = all_users_total['bdate'].str.extract(r"(\d{4})").fillna("Unfiled_bdate")

all_users_total

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 4.77 µs


,id,sex,country,city,bdate
0,657980277,Male,Украина,Запорожье,1996
1,657677380,Male,Россия,Санкт-Петербург,1994
2,657433879,Female,Unfiled_country,Unfiled_city,Unfiled_bdate
3,655125877,Male,Unfiled_country,Unfiled_city,Unfiled_bdate
4,652396719,Female,Unfiled_country,Unfiled_city,1996
...,...,...,...,...,...
34767,16832,Male,Россия,Санкт-Петербург,Unfiled_bdate
34768,16506,Male,Россия,Москва,1988
34769,14605,Male,Россия,Санкт-Петербург,1988
34770,9486,Male,Россия,Москва,Unfiled_bdate


In [67]:
# получим уникальное кол-во пользователей

all_users_unique = all_users_total.drop_duplicates(subset='id')

all_users_unique

,id,sex,country,city,bdate
0,657980277,Male,Украина,Запорожье,1996
1,657677380,Male,Россия,Санкт-Петербург,1994
2,657433879,Female,Unfiled_country,Unfiled_city,Unfiled_bdate
3,655125877,Male,Unfiled_country,Unfiled_city,Unfiled_bdate
4,652396719,Female,Unfiled_country,Unfiled_city,1996
...,...,...,...,...,...
34767,16832,Male,Россия,Санкт-Петербург,Unfiled_bdate
34768,16506,Male,Россия,Москва,1988
34769,14605,Male,Россия,Санкт-Петербург,1988
34770,9486,Male,Россия,Москва,Unfiled_bdate


In [68]:
# выведем красоту

print("Уникальных пользователей групп:", all_users_total['id'].unique().shape[0])
print("Всего пользователей групп (в т.ч. дубликаты):", all_users_total.shape[0])
print("% пересечения:", round(1 - (all_users_unique['id'].shape[0] / all_users_total.shape[0]), 2))

Уникальных пользователей групп: 24203
Всего пользователей групп (в т.ч. дубликаты): 34772
% пересечения: 0.3


In [69]:
# сохраним данные

all_users_unique.to_csv("data/all_users_unique_1.txt", index=False)
all_users_total.to_csv("data/all_users_total_1.txt", index=False)

# Этап анализа полученных данных (EDA)

In [8]:
# Загрузим уникальных пользователей

df_unique = pd.read_csv("data/all_users_unique_1.txt")

In [92]:
# Распределение пользователей по странам - абсолютные значения

df_unique.value_counts(subset='country', normalize=False).round(2).head(10),df_unique.value_counts(subset='country', normalize=True).round(2).head(10)

(country
 Россия             16503
 Unfiled_country     3771
 Украина             1334
 Беларусь             988
 Казахстан            257
 США                  212
 Латвия               103
 Германия             102
 Эстония               75
 Япония                72
 dtype: int64,
 country
 Россия             0.68
 Unfiled_country    0.16
 Украина            0.06
 Беларусь           0.04
 Казахстан          0.01
 США                0.01
 Латвия             0.00
 Германия           0.00
 Эстония            0.00
 Япония             0.00
 dtype: float64)

In [93]:
# Распределение пользователей по городам РФ

df_unique[df_unique['country'] == 'Россия'].value_counts(subset='city', normalize=False).round(2).head(10) , df_unique[df_unique['country'] == 'Россия'].value_counts(subset='city', normalize=True).round(2).head(10)

(city
 Москва             3182
 Unfiled_city       2306
 Санкт-Петербург    1875
 Екатеринбург        346
 Новосибирск         338
 Нижний Новгород     306
 Краснодар           222
 Челябинск           220
 Ростов-на-Дону      219
 Воронеж             203
 dtype: int64,
 city
 Москва             0.19
 Unfiled_city       0.14
 Санкт-Петербург    0.11
 Екатеринбург       0.02
 Новосибирск        0.02
 Нижний Новгород    0.02
 Краснодар          0.01
 Челябинск          0.01
 Ростов-на-Дону     0.01
 Воронеж            0.01
 dtype: float64)

In [100]:
# Распределение пользователей по городам РФ (не учитываются пользователи, которые не указали город)

df_unique[(df_unique['country'] == 'Россия') & (df_unique['city'] != 'Unfiled_city')].value_counts(subset='city',normalize=True).round(2).head(10)

city
Москва             0.22
Санкт-Петербург    0.13
Екатеринбург       0.02
Новосибирск        0.02
Нижний Новгород    0.02
Краснодар          0.02
Челябинск          0.02
Ростов-на-Дону     0.02
Воронеж            0.01
Самара             0.01
dtype: float64

In [101]:
# Распределение пользователей из РФ по полу

df_unique[df_unique['country'] == 'Россия'].value_counts(subset='sex', normalize=False) , df_unique[df_unique['country'] == 'Россия'].value_counts(subset='sex', normalize=True).round(2)

(sex
 Male      15193
 Female     1310
 dtype: int64,
 sex
 Male      0.92
 Female    0.08
 dtype: float64)

In [102]:
# Распределение пользователей из РФ по году рождения

df_unique[df_unique['country'] == 'Россия'].value_counts(subset='bdate', normalize=True).round(2).head(10)

bdate
Unfiled_bdate    0.52
1996             0.04
1997             0.04
1994             0.04
1995             0.04
1998             0.03
1999             0.03
1993             0.03
1992             0.03
1991             0.03
dtype: float64

In [103]:
# Посмотрим детальнее на возраст пользователей из РФ, которые его указали. Высяим кол-во лет c точностью ±1 год

bdate_russia_clear = pd.DataFrame()

bdate_russia_clear = df_unique[(df_unique['country'] == 'Россия')
          & (df_unique['bdate'] != 'Unfiled_bdate')]

bdate_russia_clear['years'] = 2021 - bdate_russia_clear['bdate'].astype(int)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [104]:
bdate_russia_clear['years'].describe().round()

count    7966.0
mean       29.0
std        13.0
min        14.0
25%        23.0
50%        27.0
75%        31.0
max       119.0
Name: years, dtype: float64

In [ ]:
# Расчитаем и уберем выбросы

q1 = bdate_russia_clear['years'].quantile(0.25)
q3 = bdate_russia_clear['years'].quantile(0.75)
IQR = q3 - q1

upper = q3 + (IQR * 1.5)
lower = q1 - (IQR * 1.5)

print("Нижняя граница выбросов:", lower)
print("Верхняя граница выбросов:", upper)

In [105]:
# Расчитаем и уберем выбросы

q1 = bdate_russia_clear['years'].quantile(0.25)
q3 = bdate_russia_clear['years'].quantile(0.75)
IQR = q3 - q1

upper = q3 + (IQR * 1.5)
lower = q1 - (IQR * 1.5)

print("Нижняя граница выбросов:", lower)
print("Верхняя граница выбросов:", upper)

Нижняя граница выбросов: 11.0
Верхняя граница выбросов: 43.0


In [106]:
# Пользователи, которые указали страну Россией и указали возраст, очищенные от выбросов

bdate_russia_clear_filtered = bdate_russia_clear[(bdate_russia_clear['years'] < upper) &
                                                 (bdate_russia_clear['years'] > lower)]

In [107]:
bdate_russia_clear_filtered

,id,sex,country,city,bdate,years
1,657677380,Male,Россия,Санкт-Петербург,1994,27
10,645932548,Male,Россия,Тамбов,1988,33
11,644772513,Male,Россия,Нижневартовск,1996,25
13,642593087,Male,Россия,Москва,1993,28
15,641043090,Male,Россия,Москва,1987,34
...,...,...,...,...,...,...
24195,20665,Male,Россия,Санкт-Петербург,1981,40
24196,17576,Male,Россия,Санкт-Петербург,1989,32
24197,16839,Male,Россия,Санкт-Петербург,1989,32
24199,16506,Male,Россия,Москва,1988,33


In [112]:
# Посмотрим на меры изменчивости

bdate_russia_clear_filtered['years'].describe().round(2)  , bdate_russia_clear_filtered['years'].mode()

(count    7666.00
 mean       27.06
 std         5.09
 min        14.00
 25%        23.00
 50%        26.00
 75%        31.00
 max        42.00
 Name: years, dtype: float64,
 0    25
 dtype: int64)

In [115]:
# Итоговый возраст пользователей из россии без выбросов
bdate_russia_clear_filtered.value_counts(subset='years',
                                         normalize=False)

years
25    657
24    620
27    581
26    579
23    560
22    476
28    452
29    420
30    418
31    417
21    374
32    337
33    314
20    244
34    236
36    150
35    148
19    147
37    108
18     77
38     69
39     54
17     48
40     40
41     37
16     33
42     31
15     25
14     14
dtype: int64

In [116]:
# Превратим числовую переменную в категориальную

def range_years_func(x):
    
    if x < 18:
        return "Младше 18"
    
    if x > 17 and x < 23:
        return "18-22"    
    
    if x > 22 and x < 28:
        return "23-27"
    
    if x > 27:
        return "Старше 27"

bdate_russia_clear_filtered['range_years'] = bdate_russia_clear_filtered['years'].astype(int).apply(range_years_func)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [119]:
# Пользователей по категориям возрастов

bdate_russia_clear_filtered.value_counts(subset='range_years') , bdate_russia_clear_filtered.value_counts(subset='range_years', normalize=True).round(2)

(range_years
 Старше 27    3231
 23-27        2997
 18-22        1318
 Младше 18     120
 dtype: int64,
 range_years
 Старше 27    0.42
 23-27        0.39
 18-22        0.17
 Младше 18    0.02
 dtype: float64)

In [120]:
# Итоговый датафрейм с пользователями россии, указавших возраст и разбитых на возрастные группы.

bdate_russia_clear_filtered

,id,sex,country,city,bdate,years,range_years
1,657677380,Male,Россия,Санкт-Петербург,1994,27,23-27
10,645932548,Male,Россия,Тамбов,1988,33,Старше 27
11,644772513,Male,Россия,Нижневартовск,1996,25,23-27
13,642593087,Male,Россия,Москва,1993,28,Старше 27
15,641043090,Male,Россия,Москва,1987,34,Старше 27
...,...,...,...,...,...,...,...
24195,20665,Male,Россия,Санкт-Петербург,1981,40,Старше 27
24196,17576,Male,Россия,Санкт-Петербург,1989,32,Старше 27
24197,16839,Male,Россия,Санкт-Петербург,1989,32,Старше 27
24199,16506,Male,Россия,Москва,1988,33,Старше 27


In [121]:
# сохраним пользователей воразст/россия
bdate_russia_clear_filtered.to_csv("data/bdate_russia_clear_filtered.csv", index=False)

In [37]:
#df1 = pd.read_csv("data/bdate_russia_clear_filtered.csv")

/opt/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
